In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt #for ploting 
import seaborn as sns 
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline            

In [ ]:
df_train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')

In [ ]:
df_train.head(10)

In [ ]:
df_train.shape

In [ ]:
df_train.columns

In [ ]:
df_train.SalePrice.describe()

In [ ]:
#HISTOGRAM
sns.distplot(df_train.SalePrice)

In [ ]:
#skewness
print("Skewness: %f" % df_train.SalePrice.skew()) #It tells about the shifting of data toward the tail

In [ ]:
#Kurtosis 
print("Kurtosis: %f" % df_train.SalePrice.kurtosis())

In [ ]:
#ScatterPlot grlivarea/saleprice
data = pd.concat([df_train['SalePrice'], df_train['GrLivArea']], axis=1)
data.plot.scatter(x ='GrLivArea', y ='SalePrice', ylim=(0,800000) )

In [ ]:
#scatterplot b/w totalBsmt/saleprice
var = 'TotalBsmtSF'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000) )

Relationship with categorical features

In [ ]:
#box plot overallqual/saleprice
var = 'OverallQual'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
f, ax = plt.subplots(figsize=(8,6))
fig = sns.boxplot(x=var, y='SalePrice', data=data)
fig.axis(ymin=0, ymax=800000);

In [ ]:
#box plot YearBuilt/saleprice
var = 'YearBuilt'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
f, ax = plt.subplots(figsize=(16,8))
fig = sns.boxplot(x=var, y='SalePrice', data=data)
fig.axis(ymin=0, ymax=800000);
plt.xticks(rotation=90);   #Rotate 90 degree to x markers

In [ ]:
#correlation matrix
corrmat = df_train.corr()
f, ax = plt.subplots(figsize=(12,9))
sns.heatmap(corrmat, vmax=.8,square=True);


The above coorelation heatmap matrix is too dense to study 

In [ ]:
k=10
cols = corrmat.nlargest(k,'SalePrice')['SalePrice'].index
cm= np.corrcoef(df_train[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm,cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10},
                yticklabels=cols.values, xticklabels=cols.values)
plt.show()

# Lets plot pairplot for features with high coorelation with SalePrice

In [ ]:
sns.set()
cols = ['SalePrice','OverallQual','GrLivArea','GarageCars','TotalBsmtSF',
       'FullBath','YearBuilt']
sns.pairplot(df_train[cols], size = 10)
plt.show();

# **Filling Out Missing Data**

In [ ]:
#missing data
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent*100], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

1. It is often considered as if there is more than 15% of missing data then the feature should be deleted.
2. If there is a feature with missing data that has weak correlation with Target can also be deleted.


In [ ]:
#dealingwithmissingdata
df_train = df_train.drop((missing_data[missing_data['Total']>1]).index,1)
df_train = df_train.drop(df_train.loc[df_train['Electrical'].isnull()].index)



In [ ]:
df_train.isnull().sum().max()



# Lets solve the problem of outliers

In [ ]:
from scipy.stats import norm 
from sklearn.preprocessing import StandardScaler
from scipy import stats

In [ ]:
#standardizing Data
saleprice_scaled = StandardScaler().fit_transform(df_train['SalePrice'][:,np.newaxis]);
low_range = saleprice_scaled[saleprice_scaled[:,0].argsort()][:10]
high_range = saleprice_scaled[saleprice_scaled[:,0].argsort()][-10:]
print('outer range (low) of distribution:')
print(low_range)
print('\nouter range(high) of distribution:')
print(high_range)

    


as it can be observe that low range is close to zero but high range varies from 1 to 7

Using Bivariate 

In [ ]:
#bivariate analysis saleprice/grlivarea
var = 'GrLivArea'
data = pd.concat([df_train['SalePrice'],df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

The two outliers are the one with 7.


In [ ]:
#deleting points
df_train.sort_values(by = 'GrLivArea', ascending=False)[:2]
df_train = df_train.drop(df_train[df_train['Id']== 1299].index)
df_train = df_train.drop(df_train[df_train['Id']== 524].index)

In [ ]:
#bivariate analysis saleprice/TotalBsmtSF
var = 'TotalBsmtSF'
data = pd.concat([df_train['SalePrice'],df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0,800000));

Removing (TotalBsmtSF > 3000) is optional  and not to be worried about

# Final Step for Target Based Cleaning

In [ ]:
#histo and normalprobablity plot
sns.distplot(df_train['SalePrice'], fit=norm)
fig = plt.figure()
res = stats.probplot(df_train['SalePrice'], plot=plt);

In Case of positive skewness log transformation works


In [ ]:
#apply log transformation
df_train['SalePrice'] = np.log(df_train['SalePrice'])

In [ ]:
#transformed histo and normalprobablity plot
sns.distplot(df_train['SalePrice'], fit=norm)
fig = plt.figure()
res = stats.probplot(df_train['SalePrice'], plot=plt);

In [ ]:
#histo and normalprobablity plot on GrLivArea
sns.distplot(df_train['GrLivArea'], fit=norm)
fig = plt.figure()
res = stats.probplot(df_train['GrLivArea'], plot=plt);

In [ ]:
#transformation 
df_train['GrLivArea'] = np.log(df_train['GrLivArea'])

In [ ]:
#transformed histo and normalprobablity plot on GrLivArea
sns.distplot(df_train['GrLivArea'], fit=norm)
fig = plt.figure()
res = stats.probplot(df_train['GrLivArea'], plot=plt);

In [ ]:
#histo and normalprobablity plot on TotalBsmtSF
sns.distplot(df_train['TotalBsmtSF'], fit=norm)
fig = plt.figure()
res = stats.probplot(df_train['TotalBsmtSF'], plot=plt);

In [ ]:
#create column for new variable (one is enough because it's a binary categorical feature)
#if area>0 it gets 1, for area==0 it gets 0
df_train['HasBsmt'] = pd.Series(len(df_train['TotalBsmtSF']), index=df_train.index)
df_train['HasBsmt'] = 0 
df_train.loc[df_train['TotalBsmtSF']>0,'HasBsmt'] = 1

In [ ]:
#transform data
df_train.loc[df_train['HasBsmt']==1,'TotalBsmtSF'] = np.log(df_train['TotalBsmtSF'])

In [ ]:
#histogram and normal probability plot
sns.distplot(df_train[df_train['TotalBsmtSF']>0]['TotalBsmtSF'], fit=norm);
fig = plt.figure()
res = stats.probplot(df_train[df_train['TotalBsmtSF']>0]['TotalBsmtSF'], plot=plt)

Test for homoscedasticity is the dispersion of scatter plot 

In [ ]:
#scatter plot
plt.scatter(df_train['GrLivArea'],df_train['SalePrice'])

In [ ]:
#scatter plot
plt.scatter(df_train[df_train['TotalBsmtSF']>0]['TotalBsmtSF'], df_train[df_train['TotalBsmtSF']>0]
            ['SalePrice']);

In [ ]:
#convert categorical variable into dummy
df_train = pd.get_dummies(df_train)